In [1]:
import src.mylib
import os
KEY1=src.mylib.getKey(os.path.join('src','key.properties'))
print 'gokr=',KEY1['gokr']
print 'dataseoul=',KEY1['dataseoul']

gokr= sXf66NX9wb7bv6O7tj%2BGUQwoC8se7eLri6DeEShKqQAj19JzR8xl29bBfbN1L0%2BqGdQUtDRTn1hm6wSIfle4Tg%3D%3D
dataseoul= 674d7a584e6b656c35317057737a5a


* 열린데이터 홈페이지에가서 상세설명을 보고 그 양식에 맞춰서 KEY, TYPE, SERVICE, START_INDEX, END_INDEX, LINE_NUM 이라는 변수명을 알아 온다.

In [25]:
%%writefile src/ds_open_dataseoul_ex.py
# coding:utf-8
import os
import requests
import urlparse
import mylib

def doIt():
    keyPath=os.path.join(os.getcwd(), 'src', 'key.properties')
    key=mylib.getKey(keyPath)
    # (1) make params with resource IDs
    KEY=key['dataseoul']
    TYPE='json'
    SERVICE='SearchSTNBySubwayLineService'
    START_INDEX=str(1)
    END_INDEX=str(10)
    LINE_NUM=str(2)
    #params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX,LINE_NUM)
    params='/'+KEY+'/'+TYPE+'/'+SERVICE+'/'+START_INDEX+'/'+END_INDEX+'/'+LINE_NUM
    # (2) make a full url
    _url='http://openAPI.seoul.go.kr:8088/'
    url=urlparse.urljoin(_url,params)
    # (3) get data
    data=requests.get(url).text
    print data[:300]

if __name__ == "__main__":
    doIt()

Overwriting src/ds_open_dataseoul_ex.py


In [3]:
pwd

u'C:\\Users\\400T6B\\Code\\s_201511138'

In [24]:
import requests
import urlparse
print os.path.join(os.getcwd(),'src','key.properties')
_url='http://openAPI.seoul.go.kr:8088/'
KEY='dfadf'
TYPE='json'
SERVICE='SearchSTNBySubwayLineService'
START_INDEX=str(1)
END_INDEX=str(10)
LINE_NUM=str(2)
params=os.path.join(KEY,TYPE,SERVICE,START_INDEX,END_INDEX,LINE_NUM)

C:\Users\400T6B\Code\s_201511138\src\key.properties
http://openAPI.seoul.go.kr:8088/dfadf\json\SearchSTNBySubwayLineService\1\10\2


In [4]:
!dir src

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: A49F-0284

 C:\Users\400T6B\Code\s_201511138\src 디렉터리

2017-04-18  오후 05:14    <DIR>          .
2017-04-18  오후 05:14    <DIR>          ..
2017-03-30  오후 12:35               253 ds234_web_crawl_1.py
2017-03-30  오후 12:37               436 ds234_web_crawl_5.py
2017-04-03  오후 12:48               637 ds234_web_crawl_6.py
2017-04-18  오후 05:14               682 ds_open_dataseoul_ex.py
2017-04-11  오후 05:50               165 ds_save_mongo_5.js
2017-04-11  오후 05:29                79 ds_save_mongo_test.js
2017-03-28  오후 05:35               374 ds_web_crawl_ieee.py
2017-04-18  오후 04:29               143 key.properties
2017-03-27  오후 12:32               949 my.xml
2017-04-18  오후 05:08               259 mylib.py
2017-04-18  오후 05:09               681 mylib.pyc
2017-03-27  오후 12:25               210 mypage1.html
2017-03-28  오후 04:23             1,108 mypage3.html
2017-03-14  오전 01:52               282 test.py
2017-04-18  오후 04:36               144 _init_.py
2017-04-

* ! : 명령창에서 다음을 실행시켜서 ipython note북상에서 보여주십쇼

In [5]:
!python src/ds_open_dataseoul_ex.py

* 공공데이터 포탈은 직접들어가서 doc파일을 다운받아서 그것을 열어보면 필수 요청 변수명을 알 수 있다. 